#Run this before starting
!python -c "import torch; print(torch.__version__)"
!python -c "import torch; print(torch.version.cuda)"


In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv
from torch.nn import Parameter
from torch_geometric.nn.conv import MessagePassing
from torch_geometric.utils import remove_self_loops, add_self_loops, softmax
import torch.optim as optim
from torch_geometric.nn.conv import feast_conv


In [42]:
import datetime
versj = 1
today = datetime.date.today().strftime("%b-%d")
print(today)

Apr-21


In [49]:
versj =2

In [6]:
if torch.cuda.is_available():
    device_name = torch.device("cuda")
else:
    device_name = torch.device('cpu')

print("Using {}.".format(device_name))

Using cpu.


In [7]:
#data

import numpy as np
import torch
import random
from torch_geometric.data import Data, DataLoader

#Import data function
def get_data(filepath):
    data = np.loadtxt(filepath)
    x = data[:, :20]
    z = data[:, 20:40]
    m = data[:, 40:60]

    dataX = []
    for i in range(x.shape[0]):
        dataXs = np.column_stack((np.arange(20), x[i], z[i]))
        dataX.append(dataXs)

    dataY = m
    zipped = list(zip(dataX, dataY))
    np.random.shuffle(zipped)
    dataX, dataY = zip(*zipped)

    dataX = np.array(dataX)
    dataY = np.array(dataY)

    dataEdgeIndex = np.column_stack((np.arange(20)[:-1], np.arange(20)[1:]))
    dataEdgeIndex = np.vstack((dataEdgeIndex, dataEdgeIndex[:, ::-1])).T

    dataset = [Data(x=torch.from_numpy(x).float(), edge_index=torch.from_numpy(dataEdgeIndex).long(), y=torch.from_numpy(y).float()) for x, y in zip(dataX, dataY)]

    return dataset
dataset = get_data("data1.txt")

In [29]:
dataset = get_data("data1.txt")

#Getting data

print(len(dataset))
#Splitting dataset
datasetRun = dataset[:int(len(dataset)*0.90)]
datasetTest = dataset[int(len(dataset)*0.90):]

train_loader = datasetRun[:int(len(datasetRun)*0.75)]
test_loader = datasetRun[int(len(datasetRun)*0.75):]

"""train_loader = train_loader[:2000]
test_loader = test_loader[:500]
"""

#Checks

print('train_loader size: ', len(train_loader))
print('test_loader size: ', len(test_loader))
print('datasetTest size: ', len(datasetTest))


# one data object from train_loader:
graph = train_loader[0]
print(graph.x.shape)
print(graph.edge_index.shape)
print(graph.y.shape)

7678
train_loader size:  5182
test_loader size:  1728
datasetTest size:  768
torch.Size([20, 3])
torch.Size([2, 38])
torch.Size([20])


In [39]:
class ArchNN(torch.nn.Module):
    def __init__(self, in_channels, num_classes, heads, t_inv = True):
        super(ArchNN, self).__init__()
        self.fc0 = nn.Linear(in_channels, 16)
        #self.pool = nn.MaxPool1d(kernel_size=2)  # Add a pooling layer
        self.conv1 = feast_conv.FeaStConv(16, 32, heads=heads, t_inv=t_inv)
        self.conv2 = feast_conv.FeaStConv(32, 64, heads=heads, t_inv=t_inv)
        self.conv3 = feast_conv.FeaStConv(64, 128, heads=heads, t_inv=t_inv)
        #self.conv4 = feast_conv.FeaStConv(128, 256, heads=heads, t_inv=t_inv)
        #self.conv5 = feast_conv.FeaStConv(256, 512, heads=heads, t_inv=t_inv)
        #self.conv6 = feast_conv.FeaStConv(512, 256, heads=heads, t_inv=t_inv)
        #self.conv7 = feast_conv.FeaStConv(256, 128, heads=heads, t_inv=t_inv)
        self.fc1 = nn.Linear(128, 64)
        self.fc2 = nn.Linear(64, 1)

        self.reset_parameters()

    def reset_parameters(self):
        self.conv1.reset_parameters()
        self.conv2.reset_parameters()
        self.conv3.reset_parameters()
        #self.conv4.reset_parameters()
        #self.conv5.reset_parameters()
        #self.conv6.reset_parameters()
        #self.conv7.reset_parameters()

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = F.relu(self.fc0(x))
        #x = self.pool(x)
        x = F.relu(self.conv1(x, edge_index))
        x = F.relu(self.conv2(x, edge_index))
        x = F.relu(self.conv3(x, edge_index))
        #x = F.elu(self.conv4(x, edge_index))
        #x = F.elu(self.conv5(x, edge_index))
        #x = F.elu(self.conv6(x, edge_index))
        #x = F.elu(self.conv7(x, edge_index))
        x = F.relu(self.fc1(x))
        #x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        #F.log_softmax(x, dijm=1)
        x = torch.squeeze(x, dim=1)
        return x




In [38]:
import time
import torch
import torch.nn.functional as F


def print_info(info):
    message = ('Epoch: {}/{}, Duration: {:.3f}s,'
               'Train Loss: {:.4f}, Test Loss:{:.4f}').format(
                   info['current_epoch'], info['epochs'], info['t_duration'],
                   info['train_loss'], info['test_loss'])
    print(message)


def run(model, train_loader, test_loader, num_nodes, epochs, optimizer, device):

    for epoch in range(1, epochs + 1):
        t = time.time()
        train_loss = train(model, train_loader, optimizer, device)
        t_duration = time.time() - t
        test_loss = test(model, test_loader, num_nodes, device)
        eval_info = {
            'train_loss': train_loss,
            'test_loss': test_loss,
            'current_epoch': epoch,
            'epochs': epochs,
            't_duration': t_duration
        }

        print_info(eval_info)


def train(model, train_loader, optimizer, device):
    model.train()

    total_loss = 0
    for idx, data in enumerate(train_loader):
        optimizer.zero_grad()
        output = model(data.to(device))
        losses = F.l1_loss(output, data.y.to(device), reduction='none')  # Calculate MSE loss for each item in output and labels
        loss = losses.mean()  # Compute mean loss for backpropagation
        loss.backward(retain_graph=True)
        #loss = model.compute_loss(output, data.y)  # compute loss with L1 regularization
        #loss = F.nll_loss(log_probs, data.y)
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(train_loader)


def test(model, test_loader, num_nodes, device):
    model.eval()
    correct = 0
    total_loss = 0
    n_graphs = 0
    with torch.no_grad():
        for idx, data in enumerate(test_loader):
            out = model(data.to(device))
            #total_loss += F.nll_loss(out, data.y.to(device)).item()
            total_loss += F.l1_loss(out, data.y.to(device)).item()
            #pred = out.max(1)[1]
            #correct += pred.eq(data.y).sum().item()
            #n_graphs += data.num_graphs
    return total_loss / len(test_loader)
"""
def train(model, train_loader, optimizer, device):
    model.train()

    total_loss = 0
    for idx, data in enumerate(train_loader):
        optimizer.zero_grad()
        output = model(data.to(device))
        errors = torch.abs(output - data.y.to(device))
        error_percentage = (errors / (output))
        mean_error_percentage = error_percentage.mean()
        mean_error_percentage.backward(retain_graph=True)
        optimizer.step()
        total_loss += mean_error_percentage
        
    return total_loss / len(train_loader)

def test(model, test_loader, num_nodes, device):
    model.eval()
    correct = 0
    total_loss = 0
    n_graphs = 0
    with torch.no_grad():
        for idx, data in enumerate(test_loader):
            output = model(data.to(device))
            errors = torch.abs(output - data.y.to(device))
            error_percentage = (errors / (output))
            mean_error_percentage = error_percentage.mean()
            total_loss += mean_error_percentage
    return total_loss / len(test_loader)"""

'\ndef train(model, train_loader, optimizer, device):\n    model.train()\n\n    total_loss = 0\n    for idx, data in enumerate(train_loader):\n        optimizer.zero_grad()\n        output = model(data.to(device))\n        errors = torch.abs(output - data.y.to(device))\n        error_percentage = (errors / (output))\n        mean_error_percentage = error_percentage.mean()\n        mean_error_percentage.backward(retain_graph=True)\n        optimizer.step()\n        total_loss += mean_error_percentage\n        \n    return total_loss / len(train_loader)\n\ndef test(model, test_loader, num_nodes, device):\n    model.eval()\n    correct = 0\n    total_loss = 0\n    n_graphs = 0\n    with torch.no_grad():\n        for idx, data in enumerate(test_loader):\n            output = model(data.to(device))\n            errors = torch.abs(output - data.y.to(device))\n            error_percentage = (errors / (output))\n            mean_error_percentage = error_percentage.mean()\n            total_los

In [40]:
train_loader = train_loader[:1000]
test_loader = test_loader[:250]

In [43]:
#runner
num_nodes = train_loader[0].x.shape[0]
num_features = train_loader[0].x.shape[1]

model1 = ArchNN(num_features, num_nodes, heads=8).to(device_name)

optimizer = optim.Adam(model1.parameters(),
                       lr=0.0001,
                       weight_decay=0.001)


run(model1, train_loader, test_loader, num_nodes, 500, optimizer, device_name)

#versj defined at the top of the notebook
modelPath = today + "ver" + str(versj) +".pt"
torch.save(model1, modelPath)
versj += 1
print("saved as: ", modelPath)

Epoch: 1/500, Duration: 12.556s,Train Loss: 1.2818, Test Loss:1.1456
Epoch: 2/500, Duration: 11.318s,Train Loss: 0.9393, Test Loss:1.0070
Epoch: 3/500, Duration: 10.938s,Train Loss: 0.8549, Test Loss:0.9467
Epoch: 4/500, Duration: 11.018s,Train Loss: 0.8122, Test Loss:0.9100
Epoch: 5/500, Duration: 11.732s,Train Loss: 0.7815, Test Loss:0.8696
Epoch: 6/500, Duration: 11.774s,Train Loss: 0.7558, Test Loss:0.8754
Epoch: 7/500, Duration: 14.987s,Train Loss: 0.7401, Test Loss:0.8731
Epoch: 8/500, Duration: 13.404s,Train Loss: 0.7135, Test Loss:0.7996
Epoch: 9/500, Duration: 13.396s,Train Loss: 0.6957, Test Loss:0.8070
Epoch: 10/500, Duration: 13.773s,Train Loss: 0.6726, Test Loss:0.7774
Epoch: 11/500, Duration: 14.030s,Train Loss: 0.6581, Test Loss:0.7439
Epoch: 12/500, Duration: 14.383s,Train Loss: 0.6395, Test Loss:0.7347
Epoch: 13/500, Duration: 14.078s,Train Loss: 0.6210, Test Loss:0.7343
Epoch: 14/500, Duration: 13.425s,Train Loss: 0.6057, Test Loss:0.6814
Epoch: 15/500, Duration: 14.3

In [44]:
#load model

model1 = torch.load("Apr-19ver3.pt")
model2 = torch.load("Apr-19ver4.pt")
model3 = torch.load("Apr-19ver6.pt")
model4 = torch.load("Apr-20ver1.pt")

model5 = torch.load("Apr-21ver1.pt")

In [50]:
torch.save(model5.state_dict(), "Apr-21ver1_state_dict.pt")

In [51]:
tester_model = ArchNN(num_features, num_nodes, heads=8)
tester_model.load_state_dict(torch.load("Apr-21ver1_state_dict.pt"))
tester_model.eval()

ArchNN(
  (fc0): Linear(in_features=3, out_features=16, bias=True)
  (conv1): FeaStConv(16, 32, heads=8)
  (conv2): FeaStConv(32, 64, heads=8)
  (conv3): FeaStConv(64, 128, heads=8)
  (fc1): Linear(in_features=128, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=1, bias=True)
)

In [52]:
tester_model.eval()
print(tester_model(datasetTest[200]))
print(datasetTest[200].y)

tensor([-6.2420,  4.4492,  2.8024,  1.3069,  0.0440, -1.0451, -1.8944, -2.5164,
        -2.9681, -3.2060, -3.1623, -2.9077, -2.5055, -1.8187, -1.0070,  0.1389,
         1.3927,  2.8411,  4.4575,  5.9987], grad_fn=<SqueezeBackward1>)
tensor([-5.9670,  4.2831,  2.7279,  1.3215,  0.0818, -0.9755, -1.8374, -2.4932,
        -2.9348, -3.1570, -3.1570, -2.9348, -2.4932, -1.8374, -0.9755,  0.0818,
         1.3215,  2.7279,  4.2831,  5.9670])


In [48]:
#Test for new dataset
def testFinal(model, indata, device, option):
    model.eval()
    title = ''
    E_A = 0
    E_A_Ad = 0
    with torch.no_grad():
        for id, data in enumerate(indata):
          out = model(data.to(device))
          real = data.y.to(device)
          errorList = []
          error = 0
          for i in range(len(out)):
            e = ((out[i].item() - real[i].item() ) / out[i].item())*100
            errorList.append(e)

          errorListAbs = [abs(x) for x in errorList]

          if (option == 1):
            errorListAbs.sort()
            errorListAbsAd = errorListAbs[:-2]
            title = 'After removing the two worst errors'
          if (option == 2):
            errorListAbsAd = errorListAbs[8:11]
            title = 'Average adjusted total error for nodes 8, 9, 10'
          if (option == 3):
            errorListAbsAd = errorListAbs[7:13]
            title = 'Average adjusted total error for nodes 7, 8, 9, 10, 11, 12'


          E_A_el = sum(errorListAbs)/len(errorListAbs)
          E_A_Ad_el = sum(errorListAbsAd)/len(errorListAbsAd)

          E_A += E_A_el
          E_A_Ad += E_A_Ad_el
          
          print([round(x,2) for x in errorList])
          print('Avarage Error: '+ str(round(E_A_el,2))+'%, Adjusted Avarage Error: '+ str(round(E_A_Ad_el,2))+'%')
          print('Mean Square: '+ str(round(F.mse_loss(out, data.y.to(device)).item(),2))+'\n')

          #errorAvg += sum(errorListAbs)/len(errorListAbs)
    print(title)
    print('Avarage Tot. Error: '+ str(round(E_A/len(indata),2))+'%, Avarage Adjusted Tot. Error: '+ str(round(E_A_Ad/len(indata),2))+'%')

#Load test data set, if this is not done from beginning!
#datasetTest = get_data("/content/drive/MyDrive/datasets/data1test.txt")

#Run testFinal
testFinal(model5, datasetTest,  device_name, 3)


[0.16, 19.2, -7.8, -3.48, -2.49, -9.13, -63.18, 13.54, 4.55, 5.26, 2.59, 0.35, -6.0, 24.34, 9.57, 6.36, 3.36, 6.19, -2.8, -0.32]
Avarage Error: 9.53%, Adjusted Avarage Error: 5.38%
Mean Square: 0.0

[5.65, -3.52, -5.56, -11.59, -20.54, 79.14, 17.22, 7.9, 2.95, 0.46, -0.71, -2.41, -1.19, -8.77, 202.94, 2.84, 4.97, 4.72, 8.07, 12.76]
Avarage Error: 20.19%, Adjusted Avarage Error: 2.6%
Mean Square: 0.0

[0.46, 52.78, -4.35, -2.19, -1.98, -5.74, 284.76, 10.26, 4.56, 2.88, 2.01, 1.32, -8.22, 68.03, 10.28, 5.57, 3.43, 6.66, -96.52, -4.74]
Avarage Error: 28.84%, Adjusted Avarage Error: 4.87%
Mean Square: 0.0

[0.85, 22.68, -6.5, 0.35, -3.66, -5.36, -76.22, 9.7, 5.49, 2.8, 2.07, 1.67, -6.5, 18.62, 6.9, 4.27, 3.73, 6.79, -10.18, -1.78]
Avarage Error: 9.81%, Adjusted Avarage Error: 4.7%
Mean Square: 0.0

[0.75, 20.39, -8.82, -5.92, -4.76, -8.92, -96.91, 14.82, 5.37, 4.58, 1.77, 1.07, 0.19, 24.37, 10.01, 4.75, 1.75, 6.1, -7.88, -1.45]
Avarage Error: 11.53%, Adjusted Avarage Error: 4.63%
Mean Squa

In [195]:
# printing parameters

def count_parameters(model):
    """Count the number of trainable parameters in a PyTorch model."""
    return sum(p.numel() for p in model.parameters() if p.requires_grad)


# Print the number of parameters
print("Number of parameters in the model: {}".format(count_parameters(model4)))

Number of parameters in the model: 95545


In [187]:
#Test for new dataline

def testData(trainedModel, testset, line):
    trainedModel.eval()
    real = testset[line].y
    
    out = trainedModel(testset[line].to(device_name))
    Bool = ''
    print('{:<10}{:<10}{:<10}{:<10}'.format('real', 'out', '%error', 'Sign'))
    for i in range(len(out)):
        error = ((out[i].item() - real[i].item())/ out[i] )    *100
        if torch.sign(real[i]) == torch.sign(out[i]):
            Bool = 'True'
        else:
            Bool = 'False'
        real_formatted = '{:<10.3f}'.format(round(real[i].item(),5))
        out_formatted = '{:<10.3f}'.format(round(out[i].item(),5))
        error_formatted = '{:<10.2f}'.format(error)
        list_formatted = [real_formatted, out_formatted, error_formatted, Bool]
        print('{:<10}{:<10}{:<10}{:<10}'.format(*list_formatted))

#Test
testData(model4, datasetTest, 30)


real      out       %error    Sign      
-3.450    -3.584    3.75      True      
2.534     2.529     -0.18     True      
1.654     1.702     2.78      True      
0.834     0.839     0.59      True      
0.093     0.080     -15.10    True      
-0.553    -0.572    3.37      True      
-1.088    -1.081    -0.60     True      
-1.499    -1.474    -1.71     True      
-1.779    -1.757    -1.26     True      
-1.920    -1.923    0.11      True      
-1.920    -1.948    1.39      True      
-1.779    -1.813    1.89      True      
-1.499    -1.516    1.12      True      
-1.088    -1.084    -0.30     True      
-0.553    -0.567    2.46      True      
0.093     0.007     -1241.44  True      
0.834     0.764     -9.15     True      
1.654     1.680     1.55      True      
2.534     2.602     2.62      True      
3.450     3.525     2.13      True      


In [25]:
model_tester_data = "testdata.txt"
model_tester_dataset = create_dataset(model_tester_data)

print(model_tester_dataset[0].y)
tester = model(model_tester_dataset[0].to(device_name))

print(tester)

print(model_tester_dataset[1].y)
tester2 = model(model_tester_dataset[1].to(device_name))
print(tester2)

tensor([ 1.3550,  0.1630,  0.9600,  1.1790,  0.9760,  0.5080, -0.0740, -0.6390,
        -1.0780, -1.3180, -1.3180, -1.0780, -0.6390, -0.0740,  0.5080,  0.9760,
         1.1790,  0.9600,  0.1630, -1.3550])
tensor([ 1.4588,  0.0168,  0.9337,  1.0978,  0.8159,  0.3859, -0.1911, -0.7819,
        -1.2410, -1.4660, -1.4306, -1.0969, -0.5489,  0.1082,  0.5618,  0.9636,
         1.3206,  0.9643,  0.0294, -1.8978], grad_fn=<SqueezeBackward1>)
tensor([ 1.6200,  0.1130,  1.0340,  1.3030,  1.0950,  0.5870, -0.0530, -0.6770,
        -1.1640, -1.4290, -1.4290, -1.1640, -0.6770, -0.0530,  0.5870,  1.0950,
         1.3030,  1.0340,  0.1130, -1.6200])
tensor([ 1.7416, -0.0129,  1.0249,  1.2137,  0.9230,  0.4418, -0.1897, -0.8252,
        -1.3337, -1.5737, -1.4708, -1.0195, -0.3363,  0.4624,  0.9845,  1.3977,
         1.5277,  1.0012, -0.6278, -2.5273], grad_fn=<SqueezeBackward1>)
